In [1]:
import json
import random
import numpy as np
from tqdm import tqdm
from sbert import sbert
import pandas as pd

/root/anaconda3/envs/SBERT-WK/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
loading configuration file https://huggingface.co/binwang/bert-base-nli-stsb/resolve/main/config.json from cache at ./cache/741df96bc03932e0eade8d5031035bca0d2958179d96a37e973a5e1863450ccb.c5a70c2b03d4ec41c3610c8a99080e27c87d4599b28c5857597c193fc608bdfe
Model config BertConfig {
  "_name_or_path": "binwang/bert-base-nli-stsb",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden

to device


In [2]:
root = '/root/thesis/new_dataset/cosmos_anns_acm/acm_anns'
train_data = list(
    map(json.loads, open(f"{root}/train_data.json").readlines())
)
test_data = list(
    map(json.loads, open(f"{root}/test_data.json").readlines())
)

In [36]:
def remove_duplicate(dataset):
    for data in tqdm(dataset):
        seen_caption = set()
        new_list = []
        for caption in data['articles'][:1]:
            if caption['caption_modified'] not in seen_caption:
                new_list.append(caption)
                seen_caption.add(caption['caption_modified'])
        if len(new_list)==0:
            print('0',end='')
        data['articles'] = new_list
    return seen_caption
# train_data_full = train_data.copy()
remove_duplicate(train_data)

100%|██████████| 161754/161754 [00:00<00:00, 276715.69it/s]


{"PERSON received the finals' Most Valuable Player Award from N.B.A. Commissioner PERSON with Warriors guard PERSON hanging onto him."}

In [37]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-base')

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_stsb-bert-base/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_stsb-bert-base/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_stsb-bert-base/pytorch_model.bin
All model checkpoint weights were used when initia

In [44]:
# Single list of sentences - Possible tens of thousands of sentences
sentences = []
for data in tqdm(train_data):
    for caption in data['articles']:
        sentences.append(caption['caption'])

100%|██████████| 161754/161754 [00:00<00:00, 663543.59it/s]


In [41]:
import torch
def neg_cos_sim(a, b):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.neg(torch.mm(a_norm, b_norm.transpose(0, 1)))

In [45]:
# paraphrases = util.paraphrase_mining(model, sentences, top_k=1, score_function=neg_cos_sim)
paraphrases = util.paraphrase_mining(model, sentences, top_k=1,show_progress_bar=True,
batch_size=32,score_function=neg_cos_sim)

Batches:   0%|          | 0/5055 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 39.41 GiB total capacity; 1.49 GiB already allocated; 11.56 MiB free; 1.56 GiB reserved in total by PyTorch)

In [ ]:
furthest_list = np.empty((len(sentences),),dtype=int)
for entry in paraphrases:
    furthest_list[entry[1]]=entry[2]
sentences_dict = dict()
for x in range(len(sentences)):
    sentences_dict[sentences[x]] = x

In [3]:
def gen_negative_case_3(data, train_data, furthest_list, sentences_dict, sentences):
    # pick 1 random correct caption
    cap1 = data['articles'][np.random.randint(len(data['articles']))]
    # then find its furthest distance caption
    cap1_index = sentences_dict[cap1]
    try:
        cap2 = sentences[furthest_list[cap1_index]]
    except:
        # data_random = train_data[np.random.randint(len(train_data))]
        # while data['img_local_path'] == data_random['img_local_path']:
        #     data_random = train_data[np.random.randint(len(train_data))]
        # cap2 = data_random['articles'][np.random.randint(len(data_random['articles']))]
        return None
    
    return [data['img_local_path'],cap1,cap2,True]

In [4]:
# Take 2 random correct caption
def gen_positive_case(data):
    caption_1_idx, caption_2_idx = random.sample(range(0,len(data['articles'])),2)
    cap1 = data['articles'][caption_1_idx]['caption_modified']
    cap2 = data['articles'][caption_2_idx]['caption_modified']
    sb = sbert([data['articles'][caption_1_idx]['caption'], data['articles'][caption_2_idx]['caption']])
    if cap1 == cap2:
        return None
    return [data['img_local_path'],cap1,cap2,False,data['maskrcnn_bboxes'],sb]

In [5]:
# Take 2 random false caption
def gen_negative_case_1(data, train_data):
    data_random1 = train_data[np.random.randint(len(train_data))]
    while data['img_local_path'] == data_random1['img_local_path']:
        data_random1 = train_data[np.random.randint(len(train_data))]
    cap1 = data_random1['articles'][np.random.randint(len(data_random1['articles']))]
    data_random2 = train_data[np.random.randint(len(train_data))]
    while data['img_local_path'] == data_random2['img_local_path']:
        data_random2 = train_data[np.random.randint(len(train_data))]
    cap2 = data_random2['articles'][np.random.randint(len(data_random2['articles']))]
    sb = sbert([cap1['caption'],cap2['caption']])

    return [data['img_local_path'],cap1['caption_modified'],cap2['caption_modified'],True,data['maskrcnn_bboxes'],sb]

# Take 1 random correct caption, 1 random false caption
def gen_negative_case_2(data, train_data):
     # Pick 1st correct caption
    if np.random.rand()>0.5:
        cap1 = data['articles'][np.random.randint(len(data['articles']))]
        # Pick a random article then pick its first caption
        data_random = train_data[np.random.randint(len(train_data))]
        while data['img_local_path'] == data_random['img_local_path']:
            data_random = train_data[np.random.randint(len(train_data))]
        cap2 = data_random['articles'][np.random.randint(len(data_random['articles']))]
    else:
        cap2 = data['articles'][np.random.randint(len(data['articles']))]
        # Pick a random article then pick its first caption
        data_random = train_data[np.random.randint(len(train_data))]
        while data['img_local_path'] == data_random['img_local_path']:
            data_random = train_data[np.random.randint(len(train_data))]
        cap1 = data_random['articles'][np.random.randint(len(data_random['articles']))]
    sb = sbert([cap1['caption'],cap2['caption']])
    return [data['img_local_path'],cap1['caption_modified'],cap2['caption_modified'],True,data['maskrcnn_bboxes'],sb]

In [6]:
np.random.seed(42)
train_data_sample = np.random.choice(train_data, size=int(len(train_data)*1/100))
# Not OOC cases
l = []

for data in tqdm(train_data_sample):
    if len(data['articles'])>1:
        g=gen_positive_case(data)
        if g is not None:
            l.append(g)
    z = gen_negative_case_2(data, train_data)
    if z is not None:
        l.append(z)

100%|██████████| 1617/1617 [01:00<00:00, 26.51it/s]


In [7]:
dataframe = pd.DataFrame(
    l, columns=["img_local_path", "caption1_modified","caption2_modified","context_label","maskrcnn_bboxes","bert_base_score"],
)

In [8]:
dataframe["context_label"].value_counts()

True     1617
False     596
Name: context_label, dtype: int64

In [24]:
s = dataframe.to_json(orient='records').replace('},','}\n')[1:-1]

In [25]:
with open(f"{root}/test_data_2.json", "w") as text_file:
    text_file.write(s)

In [11]:
dataframe['predict'] =  (dataframe['bert_base_score'] < 0.5)
confusion_matrix = pd.crosstab(dataframe['predict'], dataframe['context_label'], rownames=['Predicted'], colnames=['Actual'])
print(confusion_matrix)
result = (confusion_matrix[0][0]+confusion_matrix[1][1])/confusion_matrix.sum().sum()
print('accuracy:', result)

Actual     False  True
Predicted             
False        352     0
True         244  1617
accuracy: 0.8897424310890194
